In [21]:
import sklearn
import pandas as pd
import xgboost 
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

train_data = pd.read_csv('trainms.csv')
for column in train_data.columns:
    if train_data[column].dtype == type(object):
        le = LabelEncoder()
        train_data[column] = le.fit_transform(train_data[column].astype(str))
# print(train_data.head(5))
classes = train_data.loc[:,"treatment"]
print(train_data)
y = train_data['treatment']
train_data.pop('treatment').values
X = train_data.iloc[:,:-1].values

# encoder = LabelEncoder()
# y = encoder.fit_transform(classes)


test_data = pd.read_csv('testms.csv')
for column in test_data.columns:
    if test_data[column].dtype == type(object):
        le = LabelEncoder()
    
        test_data[column] = le.fit_transform(test_data[column].astype(str))
X_test = test_data.iloc[:,:-1].values

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0, random_state=27)

SVC_model = SVC()
# KNN model requires you to specify n_neighbors,
# the number of points the classifier will look at to determine what class a new point belongs to
KNN_model = KNeighborsClassifier(n_neighbors=5)
LR_model = LogisticRegression()
RF_model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')

SVC_model.fit(X, y)
KNN_model.fit(X, y)
LR_model.fit(X,y)
RF_model.fit(X, y)

SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
LR_prediction = LR_model.predict(X_test)
RF_prediction = RF_model.predict(X_test)
xgb = XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb.fit(X, y)
xgb_prediction = xgb.predict(X_test)

eclf = VotingClassifier(estimators=[
    ('rf1', RF_model),('xbg',xgb)], voting='soft')
eclf.fit(X, y)
vc_prediction = eclf.predict(X_test)
# print(accuracy_score(SVC_prediction, y_test))
# print(accuracy_score(KNN_prediction, y_test))
# print(accuracy_score(LR_prediction, y_test))
#print(accuracy_score(RF_prediction, y_test))
prediction = pd.DataFrame(vc_prediction, columns=['treatment'])
prediction.index+=1
prediction.treatment.replace((0,1),('No','Yes'),inplace=True)
prediction.to_csv('result4.csv')

     s.no  Timestamp  Age  Gender  Country  state  self_employed  \
0       1          0   37       8       39     10              2   
1       2          1   44      14       39     11              2   
2       3          2   32      18        7     45              2   
3       4          3   31      18       38     45              2   
4       5          4   31      18       39     37              2   
5       6          5   33      18       39     36              2   
6       7          6   35       8       39     18              2   
7       8          7   39      14        7     45              2   
8       9          8   42       8       39     10              2   
9      10          9   23      18        7     45              2   
10     11         10   31      18       39     29              2   
11     12         11   29      34        6     45              2   
12     13         12   42      30       39      2              2   
13     14         13   36      18       39      

C:\Users\manish kumar\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\manish kumar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
